<a href="https://colab.research.google.com/github/soraochi1626-cmd/economics-study/blob/main/econometrics/multiple_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install py4macro
!pip install wooldridge

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 48.7 MB/s eta 0:00:00


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf
import py4macro
import wooldridge

from numba import njit
from pandas.plotting import scatter_matrix
from scipy.stats import norm, uniform, gaussian_kde, multivariate_normal
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

# 警告メッセージを非表示
import warnings
warnings.filterwarnings("ignore")

#説明
説明変数が複数の重回帰分析(Multiple Regression)

$$y_i = \beta_0 + \sum_{j=1}^{k}{\beta_j}{x_{ij}} + u_i$$

- $i = 1, 2, …, n$：観測地のインデックス
- $i = 1, 2, …, k$：説明変数の数
(注意)

$x_1$と書く場合, $x_{i1}, x_{i1},\dots, x_{n1}$を表す(第一の説明変数)。

一般的に$x_j$と書く場合, $x_{ij}, x_{ij},\dots, x_{nj}$を表す(第j番目の説明変数)。





#statsmodelsを使う

##推定
単回帰分析を使った同じデータセットwage1を使い, 賃金と教育の関係を再考する。前章と異なる点は, 複数の説明変数を導入し重回帰分析を行うことである。追加的な説明変数として次の変数を加える。
- exper：潜在的な経験年数(=年齢ー再考学歴までの年数−６)
- tenure：現在の職場での在職年数
statsmodelsを使う上で単価域となる点は, それぞれの説明変数の前に+を使いするだけである。

まず必要な変数だけを取り出し変数wage1に割り当てる。

In [4]:
wage1 = wooldridge.data('wage1').loc[:,['wage', 'educ', 'tenure', 'exper']]

In [5]:
wooldridge.data('wage1',description=True)

name of dataset: wage1
no of variables: 24
no of observations: 526

+----------+---------------------------------+
| variable | label                           |
+----------+---------------------------------+
| wage     | average hourly earnings         |
| educ     | years of education              |
| exper    | years potential experience      |
| tenure   | years with current employer     |
| nonwhite | =1 if nonwhite                  |
| female   | =1 if female                    |
| married  | =1 if married                   |
| numdep   | number of dependents            |
| smsa     | =1 if live in SMSA              |
| northcen | =1 if live in north central U.S |
| south    | =1 if live in southern region   |
| west     | =1 if live in western region    |
| construc | =1 if work in construc. indus.  |
| ndurman  | =1 if in nondur. manuf. indus.  |
| trcommpu | =1 if in trans, commun, pub ut  |
| trade    | =1 if in wholesale or retail    |
| services | =1 if in services indus.  

被説明変数に対数賃金を使うために, 新たなwage_logを追加する。

In [6]:
wage1['wage_lig'] = np.log(wage1['wage'])

回帰式では追加の説明変数の前に+をつける。

In [16]:
formula_1 = 'wage_lig ~ educ + tenure + exper'

すいて結果を変数res_1に割り当てる

In [17]:
res_1 = smf.ols(formula_1, data=wage1).fit()

次のコードで結果を表にまとめて表示することができる。

In [18]:
print(res_1.summary())

                            OLS Regression Results                            
Dep. Variable:               wage_lig   R-squared:                       0.316
Model:                            OLS   Adj. R-squared:                  0.312
Method:                 Least Squares   F-statistic:                     80.39
Date:                Sun, 04 Jan 2026   Prob (F-statistic):           9.13e-43
Time:                        05:44:28   Log-Likelihood:                -313.55
No. Observations:                 526   AIC:                             635.1
Df Residuals:                     522   BIC:                             652.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2844      0.104      2.729      0.0

#属性とメソッド
推定結果res_1の属性とメソッドdir()やpy4macro.see()で確認できる。以下ではpy4macro.see()を使う。

In [19]:
py4macro.see(res_1)

.HC0_se                   .HC1_se                   .HC2_se                   .HC3_se
.aic                      .bic                      .bse                      .centered_tss
.compare_f_test()         .compare_lm_test()        .compare_lr_test()        .condition_number
.conf_int()               .conf_int_el()            .cov_HC0                  .cov_HC1
.cov_HC2                  .cov_HC3                  .cov_kwds                 .cov_params()
.cov_type                 .df_model                 .df_resid                 .diagn
.eigenvals                .el_test()                .ess                      .f_pvalue
.f_test()                 .fittedvalues             .fvalue                   .get_influence()
.get_prediction()         .get_robustcov_results()  .info_criteria()          .initialize()
.k_constant               .llf                      .load()                   .model
.mse_model                .mse_resid                .mse_total                .nobs
.normalized_cov_pa

単回帰分析と同様に, 係数の推定は属性.paramsで取得できる。

In [20]:
res_1.params

,0
Intercept,0.284360
educ,0.092029
tenure,0.022067
exper,0.004121


res_1はSeriesなので, インデックスを使ってeducの係数だけを取り出す場合は次のようにする。

In [22]:
res_1.params.iloc[1]

np.float64(0.09202898676928341)

次に, メソッドpredict()を使って予測値を計算してみる。次の例では, educ, tenure, experの平均値でのnp.log(wage)を計算する。

In [23]:
wm = wage1.mean() #変数の平均の計算
wm

,0
wage,5.896103
educ,12.562738
tenure,5.104563
exper,17.017110
wage_lig,1.623268


wmはSeriesなので, 変数の行ラベルを使いwm['educ']でeducの平均値だけを取り出すことができる。

In [24]:
wm['educ']

np.float64(12.562737642585551)

predict()に渡す引数としては, 説明変数の辞書を作るのが一番簡単である。

In [27]:
z = {'educ':wm['educ'],
     'tenure':wm['tenure'],
     'exper':wm['exper']}

In [28]:
res_1.predict(z)

,0
0,1.623268


3つの変数が平均値を取る場合の平均時給の対数値は約1.62である

複数の予測値を計算するには, 辞書の値にlistやarrayを書き込めばいい。次の例では, tenureとexperを平均値に固定し, educを平均値, 平均値-1, 平均値+1に変化させて予測値を計算している。

In [29]:
educ_mean = wm['educ']
tenure_mean = wm['tenure']
exper_mean = wm['exper']

z2 = {'educ':[educ_mean-1, educ_mean, educ_mean+1],
                  'tenure':[tenure_mean]*3,
                  'exper':[exper_mean]*3}

educ_return = res_1.predict(z2)
educ_return

,0
0,1.531239
1,1.623268
2,1.715297


３つの変数が平均の値を取る場合, 平均賃金の対数値は1.623268であり, 上の計算と同じになることが確認できる。教育年数が1年減少すると予測値は1.531239であり, 教育年数が1年増えると予測値は1.715297となる。この結果を使い, 教育年数が一年変化する場合の影響を計算してみる。まず教育年数が1年減少する場合を計算する。

In [30]:
educ_return[0]-educ_return[1]

np.float64(-0.0920289867692834)

次に教育年数が1年増える場合を計算する。

In [31]:
educ_return[2] - educ_return[1]

np.float64(0.0920289867692834)

これらの数字の絶対値は同じである。この結果は偶然ではない。他の要因を固定し教育年数が1年変化（減少もしくは増加）した場合，平均時給の対数値がどれだけ変化したかを計算している。これは正しく教育の収益率である。この点を数式で確認するために，変数
の値が小さい場合（例えば，0.02），次の近似が成立することを思い出そう。
$$log(x + 1) \approx x \tag{1}$$
また教育年数が$h$の場合の平均時給を*w*(h)としよう。教育年数が一年増えた場合の教育の収益率は$\frac{w(h + 1) - w(h)}{w(h)}$となり(1)を使うと次のように書くことができる。

$$\begin{align}
\frac{w(h + 1) - w(h)}{w(h)} &\approx
log(\frac{w(h + 1) - w(h)}{w(h)} + 1) \\
&=log(\frac{w(h + 1)}{w(h)})  \tag{2}\\
&=log(w(h + 1)) - log(w(h))
\end{align}$$

最後の等号が上の計算で使った式である。h + 1をh - 1に変えると教育年数が1年減少した場合を考えることができる。

もう一点付け加えるために, 上の回帰分析の変数educの係数の解釈を思い出そう。教区年数が一年増えた場合, 平均賃金の対数値はどれだけ変化するかを示しており, 教育の収益率を表している。従って, ここで計算した値を同じになるはずである。


In [32]:
res_1.params['educ']

np.float64(0.09202898676928341)

predict()の引数を省略すると予測値を返す属性.fittedvaluesと同じ値を返す。それを確かめるために次のコードを評価してみる。

In [33]:
( res_1.predict() == res_1.fittedvalues ).all()

np.True_

Trueが返されているので、値は全て同じだと確認できた。

#変数の変換
上の重回帰分析では，回帰式のwageを対数化した変数を新たに作成し計算を行なっているが、回帰式の中で変数の変換を直接指定することもできる。以下で説明する方法は被説明変数・説明変数の両方に使用可能である。